In [8]:
from typing import BinaryIO
from pulp import *

#list of supplying facilities
facility=['f1','f2','f3']

#list of customers
customers=['c1','c2','c3','c4','c5']

#dmenad requirement at customers
demand={ 'c1':80,
         'c2':270,
         'c3':250,
         'c4':160,
         'c5':180}

#activation cost of facility location
actC={'f1':1000,
      'f2':1000,
      'f3':1000}

#supply limit quantity
supply={'f1':500,
        'f2':500,
        'f3':500}

#cost for transporting from facilty f to customer c
transcost={'f1':{'c1':4, 'c2':5, 'c3':6, 'c4':8, 'c5':10},
           'f2':{'c1':6, 'c2':4, 'c3':3, 'c4':5, 'c5':8},
           'f3':{'c1':9, 'c2':7, 'c3':4, 'c4':3, 'c5':4} }

#creating Lp problem
RA=LpProblem('ResAlloc',LpMinimize)

#creating decision variables, two type of dv are here, one is for quantity and one is for site selection
#continous variable for all possible connection between j and i with service quantity to be transported
ser_var=LpVariable.dicts('service',[(i,j) for i in facility
                                          for j in customers],0)

#binary variable for site selection at faciltiy 
use_var=LpVariable.dicts('Site',facility,0,1,LpBinary)

#objective function
RA += lpSum( actC[i]*use_var[i] for i in facility )  + lpSum( ser_var[(i,j)]*transcost[i][j] for i in facility for j in customers)

#constraint1 = getting collected at demand
for j in customers:
        RA += lpSum( ser_var[(i,j)] for i in facility    ) == demand[j] 

#constraint2 = starting from supplier
for i in facility:
        RA += lpSum( ser_var[(i,j)] for j in customers  ) <= supply[i]*use_var[i]

#constraint3 = binary limitation
for j in customers:
        for i in facility:
                RA += ser_var[(i,j)] <= demand[j]*use_var[i] 

#calling solver
RA.solve()
print('status:', LpStatus[RA.status] )

#print dv
#for var in RA.variables():
#        print(var, "=", value(var))

tol=0.001
for i in facility:
        if use_var[i].varValue > tol:
                print('establish facility',i)
      
for v in RA.variables():
        print(v.name, "=", v.varValue)

#print objective 
print("Optimalsol for RA =", value(RA.objective))

#another way


status: Optimal
establish facility f2
establish facility f3
Site_f1 = 0.0
Site_f2 = 1.0
Site_f3 = 1.0
service_('f1',_'c1') = 0.0
service_('f1',_'c2') = 0.0
service_('f1',_'c3') = 0.0
service_('f1',_'c4') = 0.0
service_('f1',_'c5') = 0.0
service_('f2',_'c1') = 80.0
service_('f2',_'c2') = 270.0
service_('f2',_'c3') = 150.0
service_('f2',_'c4') = 0.0
service_('f2',_'c5') = 0.0
service_('f3',_'c1') = 0.0
service_('f3',_'c2') = 0.0
service_('f3',_'c3') = 100.0
service_('f3',_'c4') = 160.0
service_('f3',_'c5') = 180.0
Optimalsol for RA = 5610.0
